In [1]:
import csv
import xml.etree.ElementTree as ET
import os

# Define the XML namespace
namespace = {'tei': 'http://www.tei-c.org/ns/1.0'}

input_directory = "../data/pdf_raw_input/xml_files"  

csv_data = []

for filename in os.listdir(input_directory):
    if filename.endswith('.xml'):
        # Parse the XML document
        tree = ET.parse(os.path.join(input_directory, filename))
        root = tree.getroot()
        year = filename.split('-')[0]
        level = filename.split('-')[1][1]

        # Initialize variables to store data
        Topic = ""
        Article_Name = ""
        Summary = ""

        # Create a list to store rows for CSV

        title_element = root.find('.//tei:titleStmt/tei:title', {'tei': 'http://www.tei-c.org/ns/1.0'})

        # Iterate through each 'div' element in the XML
        for div in root.findall('.//tei:div', namespace):
            # Get the 'head' element inside the current 'div'
            head_element = div.find('.//tei:head', namespace)
            
            # Rule 1: Set main_part to the text of the 'head' element in the previous 'div'
            if head_element is not None and "LEARNING OUTCOMES" in head_element.text:
                Topic = prev_head_text if prev_head_text else head_element.text
                try:
                    csv_data.pop()
                except:
                    continue
            
            # Rule 2: Set sub_part to the text of the 'head' element in the current 'div'
            
            Article_Name = head_element.text if head_element is not None else ""

            # Rule 3: Concatenate all text content within <p> elements
            Summary = ' '.join([p.text.strip() for p in div.findall('.//tei:p', namespace) if p.text is not None])
            try:
                if Topic == '' and "LEARNING OUTCOMES" in title_element.text:
                    head = title_element.text.replace(' LEARNING OUTCOMES', '')
                    Topic = head
            except:
                Topic = "N/A"
            if Article_Name!="LEARNING OUTCOMES":
                csv_data.append([level, Topic, year, Article_Name, Summary])

            # Save the 'head' text of the current 'div' for Rule 1 in the next iteration
            prev_head_text = head_element.text if head_element is not None else ""

# Write data to CSV file
with open('../data/pdf_raw_output/grobid_output.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Level', 'Topic', 'Year', 'Article_Name', 'Summary'])
    csv_writer.writerows(csv_data)

print("CSV file generated successfully.")

CSV file generated successfully.


In [2]:
import csv
import os
import re

input_directory = "../data/pdf_raw_input/pypdf_outputs"  
topics_data = []
pattern = re.compile(r"(\d{4})_level([IVXLC]+)_combined")

for filename in os.listdir(input_directory):

        # Match the pattern in the filename
        match = pattern.search(filename)
        
        year = match.group(1)
        level = match.group(2)

        file_path = os.path.join(input_directory, filename)
        with open(file_path, 'r') as file:
            text_data = file.readlines()

        # Data structure to store extracted information


        current_topic = None
        current_subheading = None
        current_content = None

        # Parsing the text
        for line in text_data:
            line = line.strip()
            if line.startswith("Topic:"):
                current_topic = line.split("Topic:")[1].strip()
                current_subheading = None
                current_content = None
            elif line.startswith("Learning Outcomes:"):
                pass  # Skip this line
            elif line.startswith("- "):
                current_content += line[2:].strip()
                
            elif line.startswith("") and not line.startswith("(For the below Learning Outcomes, The candidate should be able to: )"):
                if (current_topic and current_subheading and current_content):
                    topics_data.append({
                        "level": level,
                        "topic": current_topic,
                        "year":year,
                        "subheading": current_subheading,
                        "content": current_content
                    })
                current_content = ""
                current_subheading = line.strip()


# CSV file path
csv_file_path = '../data/pdf_raw_output/pypdf_output.csv'

# Open the CSV file for writing
with open(csv_file_path, 'w', newline='') as csvfile:
    # Create a CSV writer
    csv_writer = csv.writer(csvfile)

    # Write the header row
    csv_writer.writerow(['Level', 'Topic', 'Year', "Article_Name", "Summary"])

    # Write data to CSV
    for topic_data in topics_data:
        csv_writer.writerow([topic_data.get("level", ""), topic_data.get("topic", ""), topic_data.get("year", ""), topic_data.get("subheading", ""), topic_data.get("content", "")])


print(f"Data has been written to {csv_file_path}")


Data has been written to ../data/pdf_raw_output/pypdf_output.csv
